In [2]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [3]:
final_data = []
year = 2018
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

for week in weeks:
    url = f'https://www.espn.com/nfl/scoreboard/_/week/{week}/year/{year}/seasontype/2'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    week_data = []
    for board in soup.select('.ScoreboardScoreCell'):
        title = board.find_previous(class_='Card__Header__Title').text
        teams = [t.text for t in board.select('.ScoreCell__TeamName')]
        scores = [s.text for s in board.select('.ScoreCell__Score')] or ['-', '-']

        week_data.append((week, title, teams[0], scores[0], teams[1], scores[1]))

    final_data.extend(week_data)

df_scores_2018 = pd.DataFrame(final_data, columns=['Week', 'Date', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
df_scores_2018 = df_scores_2018.rename(columns={'Team 1': 'Away Team', 'Score 1': 'Away Score', 'Team 2': 'Home Team', 'Score 2':'Home Score'})
df_scores_2018['Final Score'] = df_scores_2018['Away Score'] + ' - ' + df_scores_2018['Home Score']
#print(df.to_markdown(index=False))
# Replace missing scores with NaN and dtype convert object to int 
df_scores_2018['Away Score'] = df_scores_2018['Away Score'].replace('-', np.nan)
df_scores_2018['Home Score'] = df_scores_2018['Home Score'].replace('-', np.nan)
df_scores_2018['Away Score'] = pd.to_numeric(df_scores_2018['Away Score'])
df_scores_2018['Home Score'] =  pd.to_numeric(df_scores_2018['Home Score'])

# Calculate total points
df_scores_2018['Total Points'] = df_scores_2018['Away Score'] + df_scores_2018['Home Score']

# Calculate winner
df_scores_2018['Winner'] = df_scores_2018.apply(lambda row: row['Away Team'] if row['Away Score'] > row['Home Score'] else row['Home Team'], axis=1)

# Calculate score differential
df_scores_2018['Away Score Differential'] = df_scores_2018['Home Score'] - df_scores_2018['Away Score']
df_scores_2018['Home Score Differential'] = df_scores_2018['Away Score'] - df_scores_2018['Home Score']
df_scores_2018['Matchup'] = df_scores_2018['Away Team'] + ' @ ' + df_scores_2018['Home Team']
df_scores_2018['Date'] = pd.to_datetime(df_scores_2018['Date'], format='%A, %B %d, %Y').dt.strftime('%Y-%m-%d')
# Use a lambda function to create the 'Home_Away_Winner' column
df_scores_2018['Home_Away_Winner'] = df_scores_2018.apply(lambda row: 'Home' 
                                                          if row['Winner'] == row['Home Team'] 
                                                          else 'Away' if row['Winner'] == row['Away Team'] else 'No Winner', axis=1)
df_scores_2018.tail()

,Week,Date,Away Team,Away Score,Home Team,Home Score,Final Score,Total Points,Winner,Away Score Differential,Home Score Differential,Matchup,Home_Away_Winner
251,17,2018-12-30,Bengals,13,Steelers,16,13 - 16,29,Steelers,3,-3,Bengals @ Steelers,Home
252,17,2018-12-30,Cardinals,24,Seahawks,27,24 - 27,51,Seahawks,3,-3,Cardinals @ Seahawks,Home
253,17,2018-12-30,Eagles,24,Redskins,0,24 - 0,24,Eagles,-24,24,Eagles @ Redskins,Away
254,17,2018-12-30,Browns,24,Ravens,26,24 - 26,50,Ravens,2,-2,Browns @ Ravens,Home
255,17,2018-12-30,Colts,33,Titans,17,33 - 17,50,Colts,-16,16,Colts @ Titans,Away


In [4]:
#import consensus data for spreads 
game_dates = list(df_scores_2018['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

spreads = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        spreads.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_spreads_2018 = pd.concat(spreads, ignore_index=True)

In [5]:
#clean matchup column 
df_spreads_2018['Matchup'] = df_spreads_2018['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_spreads_2018[['Away Team', 'Home Team']] = df_spreads_2018['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_spreads_2018['Away Team'] = df_spreads_2018['Away Team'].str.strip()
df_spreads_2018['Home Team'] = df_spreads_2018['Home Team'].str.strip()
#split to create away and home spread cols 
df_spreads_2018[['Away Spread', 'Home Spread']] = df_spreads_2018['Sides'].str.split(expand=True)
#split to create away and home consensus cols 
df_spreads_2018[['Away Consensus', 'Home Consensus']] = df_spreads_2018['Consensus'].str.split(expand=True)
# convert to dtype = float for consensus cols 
df_spreads_2018['Away Consensus'] =  df_spreads_2018['Away Consensus'].str.rstrip('%').astype(int)/100
df_spreads_2018['Home Consensus'] = df_spreads_2018['Home Consensus'].str.rstrip('%').astype(int)/100
#create consensus favorite vs. underdog cols 
df_spreads_2018['Con. Spread Favorite'] = df_spreads_2018.apply(lambda x: x['Away Team'] if x['Away Consensus'] > x['Home Consensus'] else x['Home Team'], axis=1)
df_spreads_2018['Con. Spread Underdog'] = df_spreads_2018.apply(lambda x: x['Away Team'] if x['Away Consensus'] < x['Home Consensus'] else x['Home Team'], axis=1)

# Define the dictionary mapping for replacements
replacement_dict = {'LV': 'Raiders',
                    'KC': 'Chiefs',
                    'NE': 'Patriots',
                    'GB': 'Packers',
                    'NO': 'Saints',
                    'BAL': 'Ravens',
                    'SF': '49ers',
                    'NYG': 'Giants',
                    'JAC': 'Jaguars',
                    'PIT': 'Steelers',
                    'TB': 'Buccaneers',
                    'CLE': 'Browns',
                    'IND': 'Colts',
                    'PHI': 'Eagles',
                    'ARI': 'Cardinals',
                    'WAS': 'Commanders',
                    'ATL': 'Falcons',
                    'NYJ': 'Jets',
                    'SEA': 'Seahawks',
                    'MIA': 'Dolphins',
                    'HOU': 'Texans',
                    'CAR': 'Panthers',
                    'CHI': 'Bears',
                    'CIN': 'Bengals',
                    'LAR': 'Rams',
                    'BUF': 'Bills',
                    'DET': 'Lions',
                    'DEN': 'Broncos',
                    'TEN': 'Titans',
                    'MIN': 'Vikings',
                    'LAC': 'Chargers',
                    'DAL': 'Cowboys'}
# Replace the values in the DataFrame column using map()
df_spreads_2018['Away Team'] = df_spreads_2018['Away Team'].map(replacement_dict)
df_spreads_2018['Home Team'] = df_spreads_2018['Home Team'].map(replacement_dict)
df_spreads_2018['Con. Spread Favorite'] = df_spreads_2018['Con. Spread Favorite'].map(replacement_dict)
df_spreads_2018['Con. Spread Underdog'] = df_spreads_2018['Con. Spread Underdog'].map(replacement_dict)
df_spreads_2018['Matchup'] = df_spreads_2018['Away Team'] + ' @ ' + df_spreads_2018['Home Team']
df_spreads_2018.tail()

,Matchup,Date,Consensus,Sides,Picks,Indepth,Away Team,Home Team,Away Spread,Home Spread,Away Consensus,Home Consensus,Con. Spread Favorite,Con. Spread Underdog
251,Bengals @ Steelers,2018-12-30,56% 44%,+14 -14,841 661,Details,Bengals,Steelers,+14,-14,0.56,0.44,Bengals,Steelers
252,Cowboys @ Giants,2018-12-30,55% 45%,+7.5 -7.5,834 669,Details,Cowboys,Giants,+7.5,-7.5,0.55,0.45,Cowboys,Giants
253,Colts @ Titans,2018-12-30,55% 45%,-5 +5,894 733,Details,Colts,Titans,-5,+5,0.55,0.45,Colts,Titans
254,Raiders @ Chiefs,2018-12-30,54% 46%,+14.5 -14.5,841 722,Details,Raiders,Chiefs,+14.5,-14.5,0.54,0.46,Raiders,Chiefs
255,Falcons @ Buccaneers,2018-12-30,52% 48%,-2.5 +2.5,829 774,Details,Falcons,Buccaneers,-2.5,+2.5,0.52,0.48,Falcons,Buccaneers


In [6]:
#import consensus data for spreads 
game_dates = list(df_scores_2018['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topoverunderconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

totals = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        totals.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_totals_2018 = pd.concat(totals, ignore_index=True)
df_totals_2018.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth
251,NFL Cle Bal,2018-12-30,53 % Over 47 % Under,41.0,556 496,Details
252,NFL Chi Min,2018-12-30,52 % Under 48 % Over,39.5,537 491,Details
253,NFL Ind Ten,2018-12-30,51 % Under 49 % Over,42.5,547 516,Details
254,NFL Dal Nyg,2018-12-30,51 % Over 49 % Under,38.5,533 508,Details
255,NFL Sf Lar,2018-12-30,50 % Over 50 % Under,50.0,549 544,Details


In [7]:
#clean matchup column 
df_totals_2018['Matchup'] = df_totals_2018['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_totals_2018[['Away Team', 'Home Team']] = df_totals_2018['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_totals_2018['Away Team'] = df_totals_2018['Away Team'].str.strip()
df_totals_2018['Home Team'] = df_totals_2018['Home Team'].str.strip()
# create consensus columns 
df_totals_2018['Over Consensus'] = df_totals_2018['Consensus'].str.extract(r'(\d+) % Over').astype(int)/100
df_totals_2018['Under Consensus'] = df_totals_2018['Consensus'].str.extract(r'(\d+) % Under').astype(int)/100
#create consensus favorites vs underdogs 
df_totals_2018['Con. Total Favorite'] = df_totals_2018[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] > x['Under Consensus'] else 'Under', axis=1)
df_totals_2018['Con. Total Underdog'] = df_totals_2018[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] < x['Under Consensus'] else 'Under', axis=1)
# Replace the values in the DataFrame column using map() from our previously defined replacement_dict
df_totals_2018['Away Team'] = df_totals_2018['Away Team'].map(replacement_dict)
df_totals_2018['Home Team'] = df_totals_2018['Home Team'].map(replacement_dict)
df_totals_2018['Matchup'] = df_totals_2018['Away Team'] + ' @ ' + df_totals_2018['Home Team']
df_totals_2018.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth,Away Team,Home Team,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog
251,Browns @ Ravens,2018-12-30,53 % Over 47 % Under,41.0,556 496,Details,Browns,Ravens,0.53,0.47,Over,Under
252,Bears @ Vikings,2018-12-30,52 % Under 48 % Over,39.5,537 491,Details,Bears,Vikings,0.48,0.52,Under,Over
253,Colts @ Titans,2018-12-30,51 % Under 49 % Over,42.5,547 516,Details,Colts,Titans,0.49,0.51,Under,Over
254,Cowboys @ Giants,2018-12-30,51 % Over 49 % Under,38.5,533 508,Details,Cowboys,Giants,0.51,0.49,Over,Under
255,49ers @ Rams,2018-12-30,50 % Over 50 % Under,50.0,549 544,Details,49ers,Rams,0.50,0.50,Under,Under


In [8]:
print(df_scores_2018.columns)
print(df_spreads_2018.columns)
print(df_totals_2018.columns)

Index(['Week', 'Date', 'Away Team', 'Away Score', 'Home Team', 'Home Score',
       'Final Score', 'Total Points', 'Winner', 'Away Score Differential',
       'Home Score Differential', 'Matchup', 'Home_Away_Winner'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Sides', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Total', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog'],
      dtype='object')


In [9]:
# pd.merge(df1, df2, on=['Key_Column_1', 'Key_Column_2'], how='inner')
combined_df = pd.merge(df_scores_2018[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 
                                       'Winner', 'Home_Away_Winner', 'Away Team', 'Away Score', 
                                       'Away Score Differential', 'Home Team', 'Home Score', 'Home Score Differential']], 
                       df_spreads_2018[['Matchup', 'Date', 'Away Team','Away Spread', 'Home Team', 
                                        'Home Spread', 'Away Consensus', 'Home Consensus', 
                                        'Con. Spread Favorite','Con. Spread Underdog']],
                       on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                       how='inner')

In [10]:
final_df_nfl_2018 = pd.merge(combined_df[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
                                           'Home_Away_Winner', 'Away Team', 'Away Score',
                                           'Away Score Differential', 'Home Team', 'Home Score',
                                           'Home Score Differential', 'Away Spread', 'Home Spread',
                                           'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
                                           'Con. Spread Underdog']],
                            df_totals_2018[['Matchup', 'Date', 'Away Team', 'Home Team', 'Total',
                                            'Over Consensus', 'Under Consensus',
                                            'Con. Total Favorite', 'Con. Total Underdog']],
                             on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                             how='inner')

final_df_nfl_2018['Win Marginal'] = np.abs(final_df_nfl_2018['Away Score'] - final_df_nfl_2018['Home Score'])

In [11]:
# 1) did away vs home team cover? -- Done 
# Convert 'Away Spread' and 'Home Spread' columns to numeric values
final_df_nfl_2018['Away Spread'] = pd.to_numeric(final_df_nfl_2018['Away Spread'])
final_df_nfl_2018['Home Spread'] = pd.to_numeric(final_df_nfl_2018['Home Spread'])

final_df_nfl_2018['Spread Covering Team'] = final_df_nfl_2018.apply(lambda row: row['Home Team'] 
                                                              if row['Home Score'] + row['Home Spread'] >= row['Away Score'] 
                                                              else row['Away Team'], axis=1)
final_df_nfl_2018[['Matchup', 'Final Score','Home Team', 'Home Spread', 'Home Score', 'Home Spread','Away Team', 'Away Spread', 'Spread Covering Team']].head()

,Matchup,Final Score,Home Team,Home Spread,Home Score,Home Spread,Away Team,Away Spread,Spread Covering Team
0,Falcons @ Eagles,12 - 18,Eagles,-1.0,18,-1.0,Falcons,1.0,Eagles
1,Steelers @ Browns,21 - 21,Browns,3.5,21,3.5,Steelers,-3.5,Browns
2,Bengals @ Colts,34 - 23,Colts,1.0,23,1.0,Bengals,-1.0,Bengals
3,Titans @ Dolphins,20 - 27,Dolphins,1.0,27,1.0,Titans,-1.0,Dolphins
4,49ers @ Vikings,16 - 24,Vikings,-6.0,24,-6.0,49ers,6.0,Vikings


In [12]:
# 2) Did over/under hit? -- Done

# Create "Totals Covered" column
final_df_nfl_2018['Totals Covering Line'] = final_df_nfl_2018.apply(lambda row: 'Over' 
                                          if row['Total Points'] > row['Total'] 
                                          else ('Under' if row['Total Points'] < row['Total'] else 'Push'), axis=1)

# Print the updated DataFrame
final_df_nfl_2018[['Matchup', 'Total Points', 'Total', 'Totals Covering Line']].head()

,Matchup,Total Points,Total,Totals Covering Line
0,Falcons @ Eagles,30,44.5,Under
1,Steelers @ Browns,42,41.0,Over
2,Bengals @ Colts,57,47.5,Over
3,Titans @ Dolphins,47,43.5,Over
4,49ers @ Vikings,40,46.5,Under


In [13]:
# 3) Did consensus fav vs underdog win? [Spread] -- Done

# Create "Consensus Spread Winner" column
final_df_nfl_2018['Consensus Spread Winner'] = final_df_nfl_2018.apply(lambda row: 'Majority' 
                                            if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] > 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] > 0.5))
                                            else 'Minority' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] < 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] < 0.5))
                                            else 'Push' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] == 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] == 0.5))
                                            else '', axis=1)

# Print the updated DataFrame
final_df_nfl_2018[['Away Team', 'Home Team', 'Away Consensus', 'Home Consensus', 'Consensus Spread Winner']].head()

,Away Team,Home Team,Away Consensus,Home Consensus,Consensus Spread Winner
0,Falcons,Eagles,0.51,0.49,Minority
1,Steelers,Browns,0.59,0.41,Minority
2,Bengals,Colts,0.60,0.40,Majority
3,Titans,Dolphins,0.62,0.38,Minority
4,49ers,Vikings,0.39,0.61,Majority


In [14]:
# 4) Did consensus fav vs underdog win? [Totals] -- Done
# Create "Total Consensus Winner" column
final_df_nfl_2018['Total Consensus Winner'] = final_df_nfl_2018.apply(lambda row: 'Majority' 
                                                  if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] > 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] > 0.5))
                                                  else 'Minority' if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] < 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] < 0.5))
                                                  else 'Push' if (row['Under Consensus'] == 0.5) or (row['Over Consensus'] == 0.5)
                                                  else '', axis=1)

# Print the updated DataFrame
#final_df_nfl_2018[['Matchup', 'Total Points', 'Total', 'Totals Covering Line', 'Over Consensus', 'Away Consensus', 'Total Consensus Winner']]

In [15]:
final_df_nfl_2018.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner'],
      dtype='object')

In [16]:
final_df_nfl_2018

,Week,Date,Matchup,Final Score,Total Points,Winner,Home_Away_Winner,Away Team,Away Score,Away Score Differential,...,Total,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog,Win Marginal,Spread Covering Team,Totals Covering Line,Consensus Spread Winner,Total Consensus Winner
0,1,2018-09-06,Falcons @ Eagles,12 - 18,30,Eagles,Home,Falcons,12,6,...,44.5,0.50,0.50,Under,Under,6,Eagles,Under,Minority,Push
1,1,2018-09-09,Steelers @ Browns,21 - 21,42,Browns,Home,Steelers,21,0,...,41.0,0.43,0.57,Under,Over,0,Browns,Over,Minority,Minority
2,1,2018-09-09,Bengals @ Colts,34 - 23,57,Bengals,Away,Bengals,34,-11,...,47.5,0.42,0.58,Under,Over,11,Bengals,Over,Majority,Minority
3,1,2018-09-09,Titans @ Dolphins,20 - 27,47,Dolphins,Home,Titans,20,7,...,43.5,0.29,0.71,Under,Over,7,Dolphins,Over,Minority,Minority
4,1,2018-09-09,49ers @ Vikings,16 - 24,40,Vikings,Home,49ers,16,8,...,46.5,0.40,0.60,Under,Over,8,Vikings,Under,Majority,Majority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,17,2018-12-30,Bears @ Vikings,24 - 10,34,Bears,Away,Bears,24,-14,...,39.5,0.48,0.52,Under,Over,14,Bears,Under,Majority,Majority
236,17,2018-12-30,Bengals @ Steelers,13 - 16,29,Steelers,Home,Bengals,13,3,...,45.5,0.63,0.37,Over,Under,3,Bengals,Under,Majority,Minority
237,17,2018-12-30,Cardinals @ Seahawks,24 - 27,51,Seahawks,Home,Cardinals,24,3,...,39.5,0.56,0.44,Over,Under,3,Cardinals,Over,Minority,Majority
238,17,2018-12-30,Browns @ Ravens,24 - 26,50,Ravens,Home,Browns,24,2,...,41.0,0.53,0.47,Over,Under,2,Browns,Over,Majority,Majority


In [17]:
final_df_nfl_2018.to_csv('final_df_nfl_2018.csv')